# Model exploration

In this notebook we explore how our different model components work.

In [5]:
DATA_PATH = '../data/'

## Image-to-text

In [1]:
from transformers import pipeline

# create image_to_text model
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


### Local images

In [7]:
text = image_to_text(DATA_PATH+"poem_images/0.jpg")[0]["generated_text"]
print("Description: ", text)

c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Description:  two pink flowers in a flower pot 


### Online images

Instead of a local path we can also pass an online image URL to the `image_to_text` function.

In [8]:
text = image_to_text("https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2F3.bp.blogspot.com%2F-2pq1tT_bwRQ%2FUSMp_PHAjtI%2FAAAAAAAAThM%2FPX2H_FV13w4%2Fs1600%2FMaple%2BPoetic%2BWallpaper.jpg&f=1&nofb=1&ipt=efde060abd000da0e74c17c7171a6a56909b67445a355f7e04afe2f094360b36&ipo=images")
text = text[0]["generated_text"]
print("Description: ", text)

c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Description:  [{'generated_text': 'a painting of a flower arrangement in a flowery sky '}]


### Multiple images

We can give the model a list of images to caption. The model will then caption each image individually.

In [6]:
texts = image_to_text([DATA_PATH+"poem_images/0.jpg", DATA_PATH+"poem_images/3.jpg"])
descriptions = [text[0]["generated_text"] for text in texts]
print("Descriptions: ", descriptions)

c:\Users\lukas\miniconda3\envs\nlp_venv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Descriptions:  ['two pink flowers in a flower pot ', 'a row of boats sitting on a dock ']
